<a href="https://colab.research.google.com/github/MauriDeluca/TFMDeLuca/blob/main/TFM_script_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install feedparser
!pip install git+https://github.com/huggingface/transformers.git
!pip install --upgrade tensorflow
!pip install git+https://github.com/urllib3/urllib3.git
!pip install transformers

In [ ]:
import pandas as pd
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import time
from transformers import pipeline
from tqdm import tqdm
import re

In [ ]:
dfclients = pd.read_excel(r'Destinos.xlsx')
d=dfclients['Destinos']
d.values.tolist()
d = d.replace("++","+")
series= pd.Series(d)
series = series.str.strip('+')

In [ ]:
destinos = series.values.tolist()
destinos

['Buenos+Aires',
 'La+Plata',
 'Mar+del+Plata',
 'Tandil',
 'Cordoba',
 'Mendoza',
 'Bariloche',
 'Salta',
 'Misiones',
 'Calafate']

In [ ]:
df = pd.DataFrame(customers_list, columns=['Destinos'])
df= df.dropna()
df

In [ ]:
#Scrap news
destinolist = []

class ParseFeed():
 
    def __init__(self, url):
        self.feed_url = url

    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text
 
    def parse(self):
        '''
        Parse the URL, and print all the details of the news 
        '''
        feeds = feedparser.parse(self.feed_url).entries
        for f in feeds:
              destinolist.append({
                'Description': self.clean(f.get("description", "")),
                'Published Date': f.get("published", ""),
                'Title': f.get("title", ""),
                'Url': f.get("link", "")
              })
              break
        print(destinolist)


def empty():
    destinolist[:] = []

In [ ]:
#Lists with keywords
words= ["clima","Temperatura+promedio+anual","Lugares+para+visitar","Arquitectura","Platos","Comida+tipica","Bebida+tipica","Atracciones","Oferta+hotelera","Cantidad+habitantes","Que+hacer?","en+que+epoca+ir","cuantos+dias+visitar","como+recorrer","Actividades+para+hacer","Oferta+gastronomica","Transporte","Costo+para+vacacionar","pronostico+clima" ]


In [ ]:
#creation of df in blank with columns to fill
scrap_df= pd.DataFrame(columns = ["Description", "Published Date", "Title", "Url","search_var"])
#loop first for customers, after for words and if the word is "neue" it loops with the specific words. If not is only customer-word.
for destino in destinos:
  for word in words:
    word = str(word.encode('utf-8'))[2:-1]
    url = "https://news.google.com.ar/rss/search?q="+destino+"+"+word+"&hl=es-ES&sort=date&gl=AR&num=100&output=rss"
    print(url)
    feed = ParseFeed(url)
    feed.parse()
    df=pd.DataFrame(destinolist,columns=["Description","Published Date", "Title","Url"])
    df["destino"]=destino
    df["search_var"]=destino +'+'+ word
    scrap_df=scrap_df.append(df,ignore_index=True)
    empty()
    time.sleep(0.01)

https://news.google.com.ar/rss/search?q=Buenos+Aires+clima&hl=es-ES&sort=date&gl=AR&num=100&output=rss
[{'Description': 'Clima en Buenos Aires: el pronóstico del tiempo para este viernes 12 de marzo  Página 12¿Cómo estará el clima en la Ciudad de Buenos Aires y el resto del país? Pronóstico del 12 de marzo de 2021  ámbito.comEl clima de hoy en Buenos Aires y el resto de Argentina. Pronóstico del 12 de marzo.  Bae NegociosClima en la Ciudad de Buenos Aires y alrededores: el pronóstico del tiempo para este viernes 12 de marzo  El IntransigenteJueves nublado y con una máxima de 32 grados en la Ciudad de Buenos Aires y alrededores  TélamVer cobertura completa en Google Noticias', 'Published Date': 'Fri, 12 Mar 2021 10:28:06 GMT', 'Title': 'Clima en Buenos Aires: el pronóstico del tiempo para este viernes 12 de marzo - Página 12', 'Url': 'https://www.pagina12.com.ar/329074-clima-en-buenos-aires-el-pronostico-del-tiempo-para-este-vie'}]
https://news.google.com.ar/rss/search?q=Buenos+Aires+Te

In [ ]:
scrap_df

,Description,Published Date,Title,Url,search_var,destino
0,Clima en Buenos Aires: el pronóstico del tiemp...,"Fri, 12 Mar 2021 10:28:06 GMT",Clima en Buenos Aires: el pronóstico del tiemp...,https://www.pagina12.com.ar/329074-clima-en-bu...,Buenos+Aires+clima,Buenos+Aires
1,Clima en la Ciudad de Buenos Aires y alrededor...,"Sun, 21 Feb 2021 08:00:00 GMT",Clima en la Ciudad de Buenos Aires y alrededor...,https://elintransigente.com/2021/02/clima-en-l...,Buenos+Aires+Temperatura+promedio+anual,Buenos+Aires
2,Semana Santa 2021: 5 destinos para descansar q...,"Wed, 10 Mar 2021 22:48:00 GMT",Semana Santa 2021: 5 destinos para descansar q...,https://bolavip.com/america/Semana-Santa-2021-...,Buenos+Aires+Lugares+para+visitar,Buenos+Aires
3,Diálogos en el Abasto | Noticias buenosaires....,"Thu, 25 Feb 2021 08:00:00 GMT",Diálogos en el Abasto | Noticias - buenosaires...,https://www.buenosaires.gob.ar/cultura/patrimo...,Buenos+Aires+Arquitectura,Buenos+Aires
4,Llega una nueva edición del Desafío de Cocina ...,"Fri, 12 Mar 2021 10:30:00 GMT",Llega una nueva edición del Desafío de Cocina ...,https://www.buenosaires.gob.ar/noticias/llega-...,Buenos+Aires+Platos,Buenos+Aires
...,...,...,...,...,...,...
185,Qué hacer en Calafate en otoño: viajando por l...,"Wed, 10 Mar 2021 09:51:34 GMT",Qué hacer en Calafate en otoño: viajando por l...,https://conocedores.com/que-hacer-en-calafate-...,Calafate+Actividades+para+hacer,Calafate
186,"Bariloche, Madryn y Calafate con promociones i...","Thu, 11 Mar 2021 14:10:29 GMT","Bariloche, Madryn y Calafate con promociones i...",https://www.cronista.com/clase/break/semana-sa...,Calafate+Oferta+gastronomica,Calafate
187,La operadora Piamonte lanzó en Argentina circu...,"Fri, 12 Mar 2021 02:38:40 GMT",La operadora Piamonte lanzó en Argentina circu...,https://ahoracalafate.com.ar/contenido/4921/la...,Calafate+Transporte,Calafate
188,"Bariloche, Madryn y Calafate con promociones i...","Thu, 11 Mar 2021 14:10:29 GMT","Bariloche, Madryn y Calafate con promociones i...",https://www.cronista.com/clase/break/semana-sa...,Calafate+Costo+para+vacacionar,Calafate


In [ ]:
#duplicates
scrap_df= scrap_df.drop_duplicates(subset='Title', keep='first')
len(scrap_df)

146

In [ ]:
classifier = pipeline("zero-shot-classification")
listadonoticias=[]
candidate_labels = ["relaunch", "wiedereröffnung","neueröffnung","event","skandal","app","app+launch","innovation","sponsor","entlassung","vorstand","geschäftsführer","jubiläum","personal","image","konkurrenz","kampagne","marketingoffensive","branding","investition","initiative", "verkauf", "werben", "entwicklung", "trennung","markenbotschafter", "leitung","mergers+acquisition","neue produkt","neue agentur","neue marketing+manager","neue webseite","neue filiale","neue standort","neue fabrik", "neue design","neue corporate+design","neue modell","neue marke", "neue standort"]
candidate_results = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

for i in tqdm(scrap_df['Title'].values):
    # To do multi-class classification, simply pass multi_class=True.
    # In this case, the scores will be independent, but each will fall between 0 and 1.
    res = classifier(i, candidate_labels)
    if res['scores'][0] > 0.20:
        listadonoticias.append([i,res['labels'][0],res['scores'][0]])
        

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartModel: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 192/192 [1:24:12<00:00, 26.32s/it]


In [ ]:
print(listadonoticias)

[['Virtueller Basketball-Allstar Day: Premiere auf Streaming-Plattform Twitch - Sportbuzzer', 'event', 0.27369770407676697], ['Corona live: Drosten – So schnell wie möglich breit impfen – auch mit Astrazeneca - DIE WELT', 'konkurrenz', 0.20580735802650452], ['"Handfester Skandal": Kritik an Präsenzparteitag der CSU München - BR24', 'skandal', 0.5049407482147217], ['Das müssen Sie über den Corona-Impfstoff von Astrazeneca wissen - BR24', 'sponsor', 0.21438200771808624], ['Projekte mit 57 MW Gesamtleistung mit Kronos Solar unterzeichnet: BELECTRIC erhält Auftrag zur Errichtung von drei Solaranlagen in den Niederlanden – Lokale Nachrichten aus Stadt und Landkreis Schweinfurt - inUNDumSCHWEINFURT_DE', 'sponsor', 0.3570168614387512], ['Herzogin Meghan: Jetzt hat auch ihre Wachsfigur einen Babybauch - RTL Online', 'neueröffnung', 0.22128427028656006], ['Covid-19 - Impfstoff von Astrazeneca: Berichte über Nebenwirkungen und Akzeptanzprobleme - Deutschlandfunk', 'neue agentur', 0.2102175503969

In [ ]:
dfnews = pd.DataFrame(listadonoticias, columns=['Title', 'label', 'score'])

In [ ]:
scrap_df = pd.merge(scrap_df, dfnews, on='Title')

In [ ]:
scrap_df

,Description,Published Date,Title,Url,search_var,customer,label,score
0,Virtueller Basketball-Allstar Day: Premiere au...,"Wed, 17 Feb 2021 15:09:00 GMT",Virtueller Basketball-Allstar Day: Premiere au...,https://www.sportbuzzer.de/artikel/virtueller-...,EWE+event,EWE,event,0.273698
1,Corona live: Drosten – So schnell wie möglich ...,"Tue, 16 Feb 2021 23:48:15 GMT",Corona live: Drosten – So schnell wie möglich ...,https://www.welt.de/vermischtes/live221095606/...,Bayerischer+Rundfunk+event,Bayerischer+Rundfunk,konkurrenz,0.205807
2,"""Handfester Skandal"": Kritik an Präsenzparteit...","Tue, 16 Feb 2021 18:55:00 GMT","""Handfester Skandal"": Kritik an Präsenzparteit...",https://www.br.de/nachrichten/bayern/handfeste...,Bayerischer+Rundfunk+skandal,Bayerischer+Rundfunk,skandal,0.504941
3,Das müssen Sie über den Corona-Impfstoff von A...,"Wed, 17 Feb 2021 15:04:00 GMT",Das müssen Sie über den Corona-Impfstoff von A...,https://www.br.de/nachrichten/wissen/das-mcoro...,Bayerischer+Rundfunk+image,Bayerischer+Rundfunk,sponsor,0.214382
4,Projekte mit 57 MW Gesamtleistung mit Kronos S...,"Wed, 17 Feb 2021 12:20:06 GMT",Projekte mit 57 MW Gesamtleistung mit Kronos S...,https://in-und-um-schweinfurt.de/business/proj...,Vattenfall+entwicklung,Vattenfall,sponsor,0.357017
5,Herzogin Meghan: Jetzt hat auch ihre Wachsfigu...,"Tue, 16 Feb 2021 21:49:51 GMT",Herzogin Meghan: Jetzt hat auch ihre Wachsfigu...,https://www.rtl.de/cms/herzogin-meghan-jetzt-h...,Madame+Tussauds+Deutschland+personal,Madame+Tussauds+Deutschland,neueröffnung,0.221284
6,Covid-19 - Impfstoff von Astrazeneca: Berichte...,"Tue, 16 Feb 2021 23:05:43 GMT",Covid-19 - Impfstoff von Astrazeneca: Berichte...,https://www.deutschlandfunk.de/covid-19-impfst...,Westdeutscher+Rundfunk+vorstand,Westdeutscher+Rundfunk,neue agentur,0.210218
7,„TV Spielfilm“ & Co.: Reise durchs Fernsehprog...,"Wed, 17 Feb 2021 16:25:33 GMT",„TV Spielfilm“ & Co.: Reise durchs Fernsehprog...,https://www.tagesspiegel.de/gesellschaft/medie...,Westdeutscher+Rundfunk+konkurrenz,Westdeutscher+Rundfunk,relaunch,0.236866
8,Lebensmittelrettung in Gnutz: Mission gegen di...,"Wed, 17 Feb 2021 11:37:00 GMT",Lebensmittelrettung in Gnutz: Mission gegen di...,https://www.kn-online.de/Lokales/Rendsburg/Leb...,WWF+Deutschland+app,WWF+Deutschland,leitung,0.230436
9,USA: Donald Trump befeuert Machtkampf bei den ...,"Wed, 17 Feb 2021 07:00:00 GMT",USA: Donald Trump befeuert Machtkampf bei den ...,https://www.ksta.de/politik/us-blog-trump-befe...,Suedwestrundfunk+entlassung,Suedwestrundfunk,kampagne,0.246645


In [ ]:
#export to a excel
scrap_df.to_excel("draftdestinos.xlsx", encoding='utf-8')